<a href="https://colab.research.google.com/github/talhaanwarch/Offensive-Language-Detection/blob/master/OLID_Classification_biGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OLID

/content/drive/My Drive/dataset/OLID


In [3]:
ls

glove.6B.100d.txt  labels-levela.csv       testset-levela.tsv
glove.6B.200d.txt  olid-training-v1.0.tsv


In [4]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
train=pd.read_csv( 'olid-training-v1.0.tsv',sep="\t")
test=pd.read_csv('testset-levela.tsv',sep="\t")
y_test=pd.read_csv( 'labels-levela.csv',header=None).iloc[:,-1]
#OFF=0 
#NOT=1

In [6]:
y_train=train['subtask_a']
train=train['tweet']
test=test['tweet']
y_train=pd.factorize(y_train)[0]
y_test=pd.factorize(y_test)[0]

import collections
collections.Counter(y_train)

#Counter({0: 4400, 1: 8840})

Counter({0: 4400, 1: 8840})

In [0]:
# Creating the training corpus
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()
corpus_train = []
for i in train:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    x = [word for word in x if not word in stop_words]
    x=" ".join(x)
    corpus_train.append(x)    
# Creating the training corpus
corpus_test = []
for i in test:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    x = [word for word in x if not word in stop_words]
    x=" ".join(x)
    corpus_test.append(x) 

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(corpus_train)
sequences = tokenize.texts_to_sequences(corpus_train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [0]:

from keras.models import Sequential
from keras.layers import Embedding,CuDNNLSTM,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
model = Sequential()
model.add(Embedding(input_dim=max_words,output_dim=100,input_length=max_len))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences = True)))
model.add(Bidirectional(CuDNNLSTM(30, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="relu"))

model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,verbose=2,class_weight=class_weights)

Epoch 1/10
 - 21s - loss: 0.6341 - acc: 0.6117
Epoch 2/10
 - 19s - loss: 0.4481 - acc: 0.8093
Epoch 3/10
 - 19s - loss: 0.3438 - acc: 0.8621
Epoch 4/10
 - 19s - loss: 0.2648 - acc: 0.9004
Epoch 5/10
 - 19s - loss: 0.2114 - acc: 0.9230
Epoch 6/10
 - 18s - loss: 0.1759 - acc: 0.9364
Epoch 7/10
 - 18s - loss: 0.1369 - acc: 0.9527
Epoch 8/10
 - 18s - loss: 0.1168 - acc: 0.9591
Epoch 9/10
 - 18s - loss: 0.0981 - acc: 0.9666
Epoch 10/10
 - 18s - loss: 0.0835 - acc: 0.9705


In [12]:
test_sequences = tokenize.texts_to_sequences(corpus_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
print(model.evaluate(test_sequences_matrix,y_test))

860/860 [==============================] - 2s 2ms/step
[1.2450604910074279, 0.746511627352515]


In [13]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_sequences_matrix, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_test, y_pred))


860/860 [==============================] - 1s 757us/step
              precision    recall  f1-score   support

           0       0.55      0.55      0.55       240
           1       0.83      0.82      0.82       620

    accuracy                           0.75       860
   macro avg       0.69      0.69      0.69       860
weighted avg       0.75      0.75      0.75       860



In [0]:
import keras
keras.backend.clear_session()